In [13]:
import os              
os.environ['PYTHONHASHSEED'] = '0'
import pandas as pd                                                    
import numpy as np                                                     
import scanpy as sc                                                                                 
from time import time                                                       
import sys
import matplotlib
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from anndata import AnnData, read_h5ad, concat
from tqdm import tqdm
import scipy
import scipy.stats as ss
from openpyxl import load_workbook

In [5]:
gw15 = read_h5ad("gw15.h5ad")
gw20 = read_h5ad("gw20.h5ad")
gw22 = read_h5ad("gw22.h5ad")
gw34 = read_h5ad("gw34.h5ad")

gw15.obs['H3_annotation'] = gw15.obs['H3_annotation'].astype(str).where(gw15.obs['H3_annotation'].notna(), gw15.obs['H2_annotation'].astype(str) + '-c0')
gw20.obs['H3_annotation'] = gw20.obs['H3_annotation'].astype(str).where(gw20.obs['H3_annotation'].notna(), gw20.obs['H2_annotation'].astype(str) + '-c0')
gw22.obs['H3_annotation'] = gw22.obs['H3_annotation'].astype(str).where(gw22.obs['H3_annotation'].notna(), gw22.obs['H2_annotation'].astype(str) + '-c0')
gw34.obs['H3_annotation'] = gw34.obs['H3_annotation'].astype(str).where(gw34.obs['H3_annotation'].notna(), gw34.obs['H2_annotation'].astype(str) + '-c0')

/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [6]:
gw_lst = ["gw15", "gw20", "gw22", "gw34"]
# gw_lst = ["gw34"]
for gw in gw_lst:

    if "H3_annotation" in locals()[gw].obs.columns:
        locals()[gw].obs = locals()[gw].obs.drop(columns="H3_annotation")
    # locals()[gw].obs = locals()[gw].obs.join(shc_result_tot, how="left")
    locals()[gw].obs["H3_annotation"] = locals()[gw].obs["H2_annotation"].astype(str) + " Cluster " + locals()[gw].obs["H3_cluster"].astype(str)
    locals()[gw].obs["H3_annotation"] = "'" + locals()[gw].obs["H3_annotation"]

In [8]:
def read_xlsx(file, sheet_name=None, header=None):
    excel = pd.ExcelFile(load_workbook(file), engine="openpyxl")
    sheet_name = sheet_name or excel.sheet_names[0]
    sheet = excel.book[sheet_name]
    df = excel.parse(sheet_name, header=header)

    for item in sheet.merged_cells:
        top_col, top_row, bottom_col, bottom_row = item.bounds
        base_value = item.start_cell.value
        top_row -= 1
        top_col -= 1
        if header is not None:
            top_row -= header + 1
            bottom_row -= header + 1
        df.iloc[top_row:bottom_row, top_col:bottom_col] = base_value
    return df

en_group = read_xlsx("EN groups based on sankey.xlsx", header=0)

def rep_cluster(name):
    return name.replace("cluster", "Cluster")

key_lst = ["clusters from GW15", "clusters from GW20", "clusters from GW22", "clusters from GW34"]
data_lst = ["gw15", "gw20", "gw22", "gw34"]
# key_lst = ["clusters from GW34"]
# data_lst = ["gw34_raw"]
for i in range(len(key_lst)):
    key = key_lst[i]
    group_dict = {rep_cluster(en_group[key][i]): en_group['Group name'][i] for i in range(en_group.shape[0]) if not pd.isna(en_group[key][i]) }
    locals()[data_lst[i]].obs['group'] = locals()[data_lst[i]].obs['H3_annotation'].map(group_dict)
    locals()[data_lst[i]] = locals()[data_lst[i]][~pd.isna(locals()[data_lst[i]].obs['group'])]
    # locals()[data_lst[i]].obs = locals()[data_lst[i]].obs[['gw', 'H1_annotation', 'H2_annotation', 'H3_annotation', 'group']]
    

In [9]:
gw_scale = concat([gw15, gw20, gw22, gw34], axis=0)

/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [10]:
def extract_group(gw, group):
    if group == "'EN-IT-L5/6":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["'EN-IT-L5/6 Cluster 1", "'EN-IT-L5/6 Cluster 2",
                                       "'EN-IT-L5/6 Cluster 3", "'EN-IT-L5 Cluster 1",
                                       "'EN-IT-L5 Cluster 2", "'EN-IT-L5 Cluster 3"]))
                   & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L4":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["'EN-IT-L3/4-1 Cluster 1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L3":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["'EN-IT-L3/4-1 Cluster 1", "'EN-IT-L5/6 Cluster 1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    elif group == "'EN-IT-L2":
        gw15_sub = gw[(gw.obs.H3_annotation.isin(["'EN-IT-L3/4-1 Cluster 1"]))
                    & (gw.obs.gw == '15')]
        gw_other = gw[(gw.obs.group == group) & (gw.obs.H1_annotation != '15')]
        gw_group = concat([gw15_sub, gw_other], axis=0)
    else:
        gw_group = gw[gw.obs.group == group]
    return gw_group

In [11]:
def zs_calcu(gw, group):
    gw_group = extract_group(gw_scale, group)
    gw_zs = gw_group[ gw_group.obs['gw'] == str(gw) ]
    gw_mean = np.mean(gw_zs.X, axis=0)
    return gw_mean

In [12]:
def nz_calcu(gw, group):
    gw_group = extract_group(gw_scale, group)
    gw_group = gw_group[ gw_group.obs['gw'] == str(gw) ]
    region_lst = np.unique(gw_group.obs['region'])
    nz_lst = []
    for i in range(len(region_lst)):
        gw_region = gw_group[gw_group.obs['region'] == region_lst[i]]
        gw_min = np.min(gw_region.X, axis=0).reshape(1,-1)
        nz_i = list(np.sum(gw_region.X > gw_min, axis=0))
        nz_lst.append(nz_i)
    nz_lst = np.array(nz_lst)
    nz_lst = np.sum(nz_lst, axis=0)
    nz_lst = nz_lst / gw_group.shape[0]
    return nz_lst

In [14]:
def fit_reg(gw_zs, colname):
    gene_lst = gw_zs.columns
    coef_lst = []

    for gene in gene_lst:
        model = LinearRegression()
        fit = model.fit(np.array([1,2,3,4]).reshape(-1, 1), gw_zs[gene])
        coef = fit.coef_[0]
        coef_lst.append(coef)
    coef_table = pd.DataFrame(coef_lst)
    coef_table.index = gene_lst
    coef_table.columns = [colname]
    # coef_table = coef_table.sort_values(by=['coef'])
    return coef_table

In [24]:
os.makedirs("result", exist_ok=True)

group_lst = ["'EN-ET-L5/6", "'EN-ET-SP/L6b", "'EN-IT-L2", "'EN-IT-L3", "'EN-IT-L4", "'EN-IT-L5/6"]

for i in range(len(group_lst)):
    group = group_lst[i]
    gw15_zs = zs_calcu(15, group)
    gw20_zs = zs_calcu(20, group)
    gw22_zs = zs_calcu(22, group)
    gw34_zs = zs_calcu(34, group)
    gw_zs = pd.DataFrame(np.array([gw15_zs, gw20_zs, gw22_zs, gw34_zs]))
    gw_zs.columns = gw_scale.var.index
    gw_zs.index = ['gw15', 'gw20', 'gw22', 'gw34']
    
    gw15_nz = nz_calcu(15, group)
    gw20_nz = nz_calcu(20, group)
    gw22_nz = nz_calcu(22, group)
    gw34_nz = nz_calcu(34, group)
    gw_nz = pd.DataFrame(np.array([gw15_nz, gw20_nz, gw22_nz, gw34_nz]))
    gw_nz.columns = gw_scale.var.index
    gw_nz.index = ['gw15', 'gw20', 'gw22', 'gw34']
    
    coef_zs = fit_reg(gw_zs, colname="zs_coef")
    coef_nz = fit_reg(gw_nz, colname="nz_coef")
    
    zs_rank = ss.rankdata(coef_zs['zs_coef'])
    nz_rank = ss.rankdata(coef_nz['nz_coef'])
    gene_rank = zs_rank + nz_rank
    coef_zs['nz_coef'] = coef_nz['nz_coef']
    coef_zs['zs_rank'] = zs_rank
    coef_zs['nz_rank'] = nz_rank
    coef_zs['rank'] = gene_rank
    coef_zs = coef_zs.sort_values(by=['rank'])
    
    group_name = group
    group_name = group_name[1:len(group_name)]
    if "/" in group_name:
        group_name = group_name.replace('/','|')
    gw_zs.to_csv(f"result/{group_name}_zs.csv")
    gw_nz.to_csv(f"result/{group_name}_nz.csv")
    coef_zs.to_csv(f"result/{group_name}_rank.csv")

/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/shunzhou/opt/anaconda3/lib/python3.9/site-packages/anndata/_core/anndata.py:1828: UserWarning